# Credit Risk Resampling Techniques

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [15]:
file_path_1 = Path('Data/fatal-police-shootings-data.csv')
df1 = pd.read_csv(file_path_1)
df1.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,-122.892,45.487,True
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,-97.281,37.695,True
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,-122.422,37.763,True
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,-104.692,40.384,True


In [20]:
file_path_2 = Path('Data/body-camera.csv')
df2 = pd.read_csv(file_path_2)
df2.head()

,City,Number of Sworn Officers,Number Wearing Body Cameras,Coverage,Fairness,Section of Provision,Policy Language/ Explanation,Transparency,Section of Provision(s),Policy Language/ Explanation.1,Privacy,Section of Provision.1,Policy Language/ Explanation.2,Accountability,Section of Provision.2,Policy Language/ Explanation.3
0,Austin,2300,20,1%,No,303.4.a.b,Body camera footage may be reviewed by an offi...,No,TX SB 158 Sec. 1701.660 and Sec. 1701.661.e,"Under TX law, police ""may"" (but are not requir...",Yes,303.2.4,All footage not depicting a criminal investiga...,No,Not specified in policy,No disciplinary guidelines or statement saying...
1,Baltimore,3080,155,5%,No,"Review of Recordings Section 2,3, and 4","Officers can view their own video footage ""to ...",Yes,Maryland Public Information Act § 10-618(f) ; ...,Requested footage should be made public once a...,Unclear,Not specified in policy,While footage is clearly being captured and st...,No,Not specified in policy,No statement that failure to adhere to the pol...
2,Charlotte,1849,1849,100%,No,3.c,"""Officers may review BWC video to aid in compl...",No,J.4.a and V.F,"As per CMPD policy, any footage showing a dead...",Yes,J.7,"All footage not depicting a misdemeanor, felon...",No,II.b,"Policy states that ""any violation of this dire..."
3,Chicago,11944,30,0.3%,No,IL Law Enforcement Officer-Worn Body Camera Ac...,"""The recording officer and his or her supervis...",Yes,IL Law Enforcement Officer-Worn Body Camera Ac...,Redacted footage will be released with permiss...,Yes,IL Law Enforcement Officer-Worn Body Camera Ac...,Footage deleted after 90 days unless it has ev...,No,VIII.A,"Policy states that ""unauthorized duplicating, ..."
4,Dallas,3474,77,2%,No,3XX.06F,"""During any administrative or criminal investi...",No,TX SB 158 Sec. 1701.660 and Sec. 1701.661.e,"Under TX law, police ""may"" (but are not requir...",No,3XX.06C,Footage deleted after 90 days if it is not cat...,No,3XX.05,While a number of activities are prohibited fr...


In [24]:
df = pd.merge(df1, df2, left_on="city", right_on="City").drop('City', axis=1)
df

,id,name,date,manner_of_death,armed,age,gender,race,city,state,...,Policy Language/ Explanation,Transparency,Section of Provision(s),Policy Language/ Explanation.1,Privacy,Section of Provision.1,Policy Language/ Explanation.2,Accountability,Section of Provision.2,Policy Language/ Explanation.3
0,75,Pablo Meza,2015-01-17,shot,gun,24.0,M,H,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
1,194,Charly Leundeu Keunang,2015-03-01,shot and Tasered,unarmed,43.0,M,B,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
2,796,Aaron Valdez,2015-03-11,shot,NaN,25.0,M,H,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
3,331,Roberto Rodriguez,2015-04-08,shot,gun,39.0,M,H,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
4,495,Luis Martinez,2015-04-21,shot,knife,35.0,M,H,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,4702,Ryan Smith,2019-05-09,shot,knife,31.0,M,B,Seattle,WA,...,Its not clear whether reviewing footage in ord...,Yes,"Not specified in policy, but redacted video is...","Not specified in policy, but Seattle PD is cur...",Unclear,Not specified in policy,No policy language,No,16.091 POL-1,No discipline for officers who violate the bod...
481,5778,Shaun Lee Fuhr,2020-05-01,shot,unarmed,24.0,M,B,Seattle,WA,...,Its not clear whether reviewing footage in ord...,Yes,"Not specified in policy, but redacted video is...","Not specified in policy, but Seattle PD is cur...",Unclear,Not specified in policy,No policy language,No,16.091 POL-1,No discipline for officers who violate the bod...
482,5816,Terry Caver,2020-05-19,shot and Tasered,knife,57.0,M,B,Seattle,WA,...,Its not clear whether reviewing footage in ord...,Yes,"Not specified in policy, but redacted video is...","Not specified in policy, but Seattle PD is cur...",Unclear,Not specified in policy,No policy language,No,16.091 POL-1,No discipline for officers who violate the bod...
483,6596,Gregory Taylor,2021-02-09,shot,gun,45.0,M,B,Seattle,WA,...,Its not clear whether reviewing footage in ord...,Yes,"Not specified in policy, but redacted video is...","Not specified in policy, but Seattle PD is cur...",Unclear,Not specified in policy,No policy language,No,16.091 POL-1,No discipline for officers who violate the bod...


In [26]:
df.to_csv('Data/combined_data.csv',index=False)


In [28]:
len(df)

485

In [29]:
columns = [
    'id',
    'name',
    'date',
    'manner_of_death',
    'armed',
    'age',
    'gender',
    'race',
    'city',
    'state',
    'signs_of_mental_illness',
    'threat_level',
    'flee',
    'body_camera',
    'longitude',
    'latitude',
    'is_geocoding_exact',
    'Number of Sworn Officers',
    'Number Wearing Body Cameras',
    'Coverage',
    'Fairness',
    'Section of Provision',
    'Policy Language/ Explanation,Transparency',
    'Section of Provision(s)',
    'Policy Language/ Explanation.1,Privacy',
    'Section of Provision.1',
    'Policy Language/ Explanation.2,Accountability',
    'Section of Provision.2',
    'Policy Language/ Explanation.3'
]

target = ["body_camera"]

In [30]:

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Converting variables from string
df['Coverage'] = df['Coverage'].str.replace('%', '')
df['Coverage'] = df['Coverage'].astype('float') / 100

df['date']= pd.to_datetime(df['date'])

df['age'] = df['age'].astype('float')
df['Number of Sworn Officers'] = df['Number of Sworn Officers'].astype('float')
df['Number Wearing Body Cameras'] = df['Number Wearing Body Cameras'].astype('float')


df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,...,Policy Language/ Explanation,Transparency,Section of Provision(s),Policy Language/ Explanation.1,Privacy,Section of Provision.1,Policy Language/ Explanation.2,Accountability,Section of Provision.2,Policy Language/ Explanation.3
0,75,Pablo Meza,2015-01-17,shot,gun,24.0,M,H,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
1,194,Charly Leundeu Keunang,2015-03-01,shot and Tasered,unarmed,43.0,M,B,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
3,331,Roberto Rodriguez,2015-04-08,shot,gun,39.0,M,H,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
4,495,Luis Martinez,2015-04-21,shot,knife,35.0,M,H,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."
5,619,Jason Hendley,2015-07-06,shot,knife,29.0,M,B,Los Angeles,CA,...,"""Officers are therefore required to review BWV...",No,"Not specified in policy, but the Police Chief ...",LA Police Chief has stated that recordings won...,Unclear,Not specified in policy,No policy language,No,VII and VIII,"Unauthorized use or release of footage, or tam..."


# Split the Data into Training and Testing

In [5]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [6]:
df_bin_encode = pd.get_dummies(df, columns=["home_ownership",
                                            "verification_status",
                                            "pymnt_plan",
                                            "application_type",
                                            "hardship_flag",
                                            "debt_settlement_flag",
                                            "initial_list_status",
                                            "next_pymnt_d"])

months_num = {
    'Jan-2019': 1,
    'Feb-2019': 2,
    'Mar-2019':3
}

# Months' and Loan Status condition encoded using the dictionary values
df_bin_encode["issue_d_num"] = df_bin_encode["issue_d"].apply(lambda x: months_num[x])
# Drop the issue_d columns
df_bin_encode = df_bin_encode.drop(["issue_d"], axis=1)

df_bin_encode.head(10)

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,pymnt_plan_n,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_num
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,1,0,1,0,1,3
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,1,0,1,0,1,3
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,1,0,1,0,1,3
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,1,0,1,0,1,3
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,1,0,1,0,1,3
5,25000.0,0.1797,634.43,90000.0,low_risk,18.49,1.0,1.0,14.0,0.0,...,1,1,0,1,1,0,1,0,1,3
6,20400.0,0.2000,540.48,51315.0,low_risk,25.63,0.0,2.0,8.0,1.0,...,1,1,0,1,1,0,1,0,1,3
7,10000.0,0.1557,349.46,66000.0,low_risk,9.00,0.0,0.0,11.0,0.0,...,1,1,0,1,1,0,1,0,1,3
8,14000.0,0.0881,443.96,45000.0,low_risk,10.24,0.0,1.0,9.0,0.0,...,1,1,0,1,1,1,0,0,1,3
9,10000.0,0.1474,345.39,63295.0,low_risk,39.34,0.0,0.0,27.0,0.0,...,1,1,0,1,1,0,1,0,1,3


In [7]:
# Create our features
X = df_bin_encode.drop(columns="loan_status")


# Create our target
y = pd.DataFrame(df_bin_encode["loan_status"])

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pymnt_plan_n,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,issue_d_num
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,0.860340,0.139660,1.0,1.0,0.123879,0.876121,0.383161,0.616839,1.726172
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.346637,0.346637,0.0,0.0,0.329446,0.329446,0.486161,0.486161,0.743862
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000,0.000000,1.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,0.000000,0.000000,1.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,0.000000,1.000000,2.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,1.000000,0.000000,1.0,1.0,0.000000,1.000000,1.000000,1.000000,2.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,3.000000


In [9]:
y.describe()

,loan_status
count,68817
unique,2
top,low_risk
freq,68470


In [10]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 1)

Counter(y_train)

Counter({'loan_status': 1})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'loan_status': 1})

In [13]:
# Train the Logistic Regression model using the resampled data

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6770635552797561

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   75,    26],
       [ 6644, 10460]])

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.74      0.61      0.02      0.67      0.46       101
   low_risk       1.00      0.61      0.74      0.76      0.67      0.45     17104

avg / total       0.99      0.61      0.74      0.75      0.67      0.45     17205



### SMOTE Oversampling

In [18]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)

Counter(y_resampled)

Counter({'loan_status': 1})

In [19]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)



LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.6703961322231382

In [21]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   69,    32],
       [ 5856, 11248]])

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.68      0.66      0.02      0.67      0.45       101
   low_risk       1.00      0.66      0.68      0.79      0.67      0.45     17104

avg / total       0.99      0.66      0.68      0.79      0.67      0.45     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [23]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'loan_status': 1})

In [24]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5357683686984227

In [26]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[   68,    33],
       [10292,  6812]])

In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.40      0.01      0.52      0.28       101
   low_risk       1.00      0.40      0.67      0.57      0.52      0.26     17104

avg / total       0.99      0.40      0.67      0.57      0.52      0.26     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

Counter(y_resampled)

Counter({'loan_status': 1})

In [29]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs')
model.fit(X_resampled, y_resampled)

LogisticRegression()

In [30]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6544957348868656

In [31]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[  75,   26],
       [7416, 9688]])

In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.74      0.57      0.02      0.65      0.43       101
   low_risk       1.00      0.57      0.74      0.72      0.65      0.41     17104

avg / total       0.99      0.57      0.74      0.72      0.65      0.41     17205

